In [1]:
from pathlib import Path
import pandas as pd

from rekall.predicates import _area, _iou, during_inv, overlaps
from stsearch.stdlib import tiou
from virat_helper import load_events, LIST_CLIP_ID, EVENTTYPE

In [2]:
print(LIST_CLIP_ID)

['VIRAT_S_010000_08_000893_001024', 'VIRAT_S_010000_03_000442_000528', 'VIRAT_S_010201_02_000167_000197', 'VIRAT_S_010003_01_000111_000137', 'VIRAT_S_050000_11_001530_001576', 'VIRAT_S_010202_06_000784_000873', 'VIRAT_S_000201_01_000384_000589', 'VIRAT_S_010113_07_000965_001013', 'VIRAT_S_050000_06_000908_000970', 'VIRAT_S_000201_06_001354_001397', 'VIRAT_S_010203_02_000347_000397', 'VIRAT_S_010201_08_000705_000739', 'VIRAT_S_010205_02_000301_000345', 'VIRAT_S_010203_00_000047_000139', 'VIRAT_S_010109_04_000487_000538', 'VIRAT_S_010107_02_000282_000312', 'VIRAT_S_040001_02_001102_001530', 'VIRAT_S_050203_06_001202_001264', 'VIRAT_S_040103_07_001011_001093', 'VIRAT_S_010003_09_000779_000861', 'VIRAT_S_010001_04_000583_000646', 'VIRAT_S_050000_08_001235_001295', 'VIRAT_S_040003_01_000083_000193', 'VIRAT_S_010111_09_000981_001014', 'VIRAT_S_010113_06_000831_000896', 'VIRAT_S_050300_08_001705_001771', 'VIRAT_S_050300_05_001272_001330', 'VIRAT_S_010200_06_000702_000744', 'VIRAT_S_040000_05_

In [3]:
def parse_event_list(clip_id, event_type) -> pd.DataFrame:
    try:
        df = load_events(clip_id)
    except FileNotFoundError:
        # some clips have no events and no event file
        return pd.DataFrame()
    
    df = df[df['event_type']==event_type]
    df['x2'] = df['x1'] + df['w']
    df['y2'] = df['y1'] + df['h']
    df['t1'], df['t2'] = df['start_frame'], df['end_frame']
    
    rv = []
    for event_id in df['event_id'].unique():
        edf = df[df['event_id']==event_id]
        rv.append({
            'event_id': event_id,
            'x1': edf['x1'].min(),
            'x2': edf['x2'].max(),
            'y1': edf['y1'].min(),
            'y2': edf['y2'].max(),
            't1': edf['t1'].min(),
            't2': edf['t2'].max(),
        })
        
    return  pd.DataFrame(rv)

def parse_all_event_list(event_type) -> pd.DataFrame:
    all_df = []
    for clip_id in LIST_CLIP_ID:
        df = parse_event_list(clip_id, event_type)
        if not df.empty:
            df['clip_id'] = clip_id
            all_df.append(df)
        
    return pd.concat(all_df, ignore_index=True)
    

# gt = parse_event_list('VIRAT_S_000202_00_000000_000977', EVENTTYPE.GET_OUT_VEHICLE)

print(GT)

    event_id   x1    x2   y1   y2    t1    t2                          clip_id
0         10  383   546  146  331  1135  1271  VIRAT_S_050000_11_001530_001576
1          0  695   767  362  420   631  1022  VIRAT_S_000201_01_000384_000589
2          0  132   213  352  435   449   701  VIRAT_S_000201_06_001354_001397
3          0  686   786  353  434   572   758  VIRAT_S_000201_03_000640_000672
4          2  423   609  509  690  3340  3517                   VIRAT_S_000006
..       ...  ...   ...  ...  ...   ...   ...                              ...
92         0  786  1034  501  711  2486  2619                   VIRAT_S_000002
93         0  529   643   58  117   656   745  VIRAT_S_050203_01_000147_000223
94         1  454   583   84  154  1067  1125  VIRAT_S_050203_01_000147_000223
95         0  660   771  475  515   637   832  VIRAT_S_000200_00_000100_000171
96         0  685   767  336  379  1180  1296  VIRAT_S_000203_07_001341_001458

[97 rows x 8 columns]


In [21]:
def parse_result(result_file="getoutcar.csv"):
    df = pd.read_csv(result_file, index_col=0)
    # convert relative coord to pixel coord
    df['x1'] = df['x1'] * df['width']
    df['x2'] = df['x2'] * df['width']
    df['y1'] = df['y1'] * df['height']
    df['y2'] = df['y2'] * df['height']
    return df

In [26]:
def is_hit(ipred, igt):
    return _iou(ipred, igt) > 0.0 and overlaps()(ipred, igt) #  tiou(i1, i2)>0.1

In [59]:
# compute precision, recall, result length
def eval_prediction(prediction_file, event_type=EVENTTYPE.GET_OUT_VEHICLE):
    GT = parse_all_event_list(EVENTTYPE.GET_OUT_VEHICLE)
    prediction = parse_result(prediction_file)
    
    GT['hit'] = False
    prediction['hit'] = False

    for i, pred in prediction.iterrows():
        for j, gt in GT[GT['clip_id']==pred['clip_id']].iterrows():
            if is_hit(pred, gt):
                prediction.loc[i, ['hit']] = True
                GT.loc[j, ['hit']] = True

    GT_hit = len(GT[GT['hit']==True])
    prediction_hit = len(prediction[prediction['hit']==True])
    video_hrs = sum((prediction['t2']-prediction['t1'])/prediction['fps'])/3600

    print(f"Precision={prediction_hit}/{len(prediction)}={prediction_hit/len(prediction):.2f}")
    print(f"Recall={GT_hit}/{len(GT)}={GT_hit/len(GT):.2f}")
    print(f"Video length={video_hrs:.2f} hrs")
    print(f"Productivity={GT_hit/video_hrs} /hr")

In [71]:
eval_prediction("getoutcar.csv")

Precision=64/923=0.07
Recall=58/97=0.60
Video length=0.84 hrs
Productivity=68.68646994966942 /hr


In [65]:
eval_prediction("person_and_car.csv")

Precision=62/441=0.14
Recall=76/97=0.78
Video length=3.86 hrs
Productivity=19.663942551006155 /hr


In [37]:
# FPs
prediction[prediction['hit']==True][['clip_id', 't1','t2', 'result_size']].sort_values(by=['clip_id'])

,clip_id,t1,t2,result_size
11,VIRAT_S_000001,9602,9902,1369755
0,VIRAT_S_000002,2342,2642,1204561
6,VIRAT_S_000003,10262,10562,980611
14,VIRAT_S_000004,6962,7262,1072663
16,VIRAT_S_000004,10982,11282,3040647
...,...,...,...,...
788,VIRAT_S_050202_00_000000_000105,842,1142,4155501
789,VIRAT_S_050202_00_000000_000105,992,1292,349688
893,VIRAT_S_050203_09_001960_002083,842,1142,833477
895,VIRAT_S_050203_09_001960_002083,902,1202,2881206


In [38]:
prediction.sort_values(by=['clip_id'])

,clip_id,t1,t2,x1,x2,y1,y2,result_size,frame_count,fps,width,height,hit
12,VIRAT_S_000001,18272,18572,1528.152100,1838.406494,0.309917,101.103577,490592,20655,30,1920,1080,False
11,VIRAT_S_000001,9602,9902,857.502930,1176.180458,363.334473,637.222176,1369755,20655,30,1920,1080,True
0,VIRAT_S_000002,2342,2642,713.980808,1135.593418,475.714905,722.233894,1204561,9075,30,1920,1080,True
1,VIRAT_S_000002,3362,3662,713.980808,1094.813690,467.924774,722.233894,1168510,9075,30,1920,1080,False
2,VIRAT_S_000002,4022,4322,713.980808,1358.815935,271.311005,722.233894,3192702,9075,30,1920,1080,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194,VIRAT_S_050301_03_000933_001046,1772,2072,687.907848,859.985962,415.453341,508.034866,343431,3347,30,1920,1080,False
1193,VIRAT_S_050301_03_000933_001046,1382,1682,1419.517822,1506.178150,189.631790,247.943326,133209,3347,30,1920,1080,False
1191,VIRAT_S_050301_03_000933_001046,92,392,1419.981422,1494.470329,186.552673,248.320246,98175,3347,30,1920,1080,False
1195,VIRAT_S_050301_03_000933_001046,2312,2612,1031.993652,1104.533203,97.171082,224.154243,235598,3347,30,1920,1080,False


In [72]:
68.7/11.2

6.133928571428572